In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from itertools import cycle
import scipy.stats as stats

In [ ]:
def BoxPloting3byGroup(X, title, type):

  Co =np.where(y == 0)[0][-1]+1
  No =np.where(y == 1)[0][-1]+1
  Mi =np.where(y == 2)[0][-1]+1
  Mo =np.where(y == 3)[0][-1]+1
  Se =np.where(y == 4)[0][-1]+1

  Control = []
  NoDR = []
  Mild = []
  Moderate = []
  Severe = []
  for j in range(X.shape[1]):
    Control.append(X[:Co, j])
    NoDR.append(X[Co:No, j])
    Mild.append(X[No:Mi, j])
    Moderate.append(X[Mi:Mo, j])
    Severe.append(X[Mo:Se, j])

  fig = plt.figure(figsize =(10, 6))
  fig.set_facecolor('white')
  ax = plt.axes()
  plt.rc('figure', dpi=1200)
  if type == 0:
    ticks = ['T-'+title[:3], 'L-'+title[:3], 'C-'+title[:3]]
  else:
    ticks = ['LC-'+title[:3]+'R', 'LT-'+title[:3]+'R', 'CT-'+title[:3]+'R']
  Control_plot = plt.boxplot(Control, positions=np.array(
      np.arange(len(Control)))*4.0-1.4, widths=0.6, showfliers=False, showmeans=True,
            meanprops={"marker": "x", "markeredgecolor": '#15B01A', "markersize": "6"})
  NoDR_plot = plt.boxplot(NoDR, positions=np.array(
      np.arange(len(NoDR)))*4.0-0.7, widths=0.6, showfliers=False, showmeans=True,
            meanprops={"marker": "x", "markeredgecolor": '#0000FF', "markersize": "6"})
  Mild_plot = plt.boxplot(Mild, positions=np.array(
      np.arange(len(Mild)))*4.0+0.0, widths=0.6, showfliers=False, showmeans=True,
            meanprops={"marker": "x", "markeredgecolor": '#9A0EEA', "markersize": "6"})
  Moderate_plot = plt.boxplot(Moderate, positions=np.array(
      np.arange(len(Moderate)))*4.0+0.7, widths=0.6, showfliers=False, showmeans=True,
            meanprops={"marker": "x", "markeredgecolor": '#FFA500', "markersize": "6"})
  Severe_plot = plt.boxplot(Severe, positions=np.array(
      np.arange(len(Severe)))*4.0+1.4, widths=0.6, showfliers=False, showmeans=True,
            meanprops={"marker": "x", "markeredgecolor": '#E50000', "markersize": "6"})

  def define_box_properties(plot_name, color_code, label):
      for k, v in plot_name.items():
          plt.setp(plot_name.get(k), color=color_code)

      plt.plot([], c=color_code, label=label)
      plt.legend(ncol=1, loc='upper right')

  define_box_properties(Control_plot, '#15B01A', 'Control')
  define_box_properties(NoDR_plot, '#0000FF', 'NoDR')
  define_box_properties(Mild_plot, '#9A0EEA', 'Mild')
  define_box_properties(Moderate_plot, '#FFA500', 'Moderate')
  define_box_properties(Severe_plot, '#E50000', 'Severe')

  # set the x label values
  plt.xticks(np.arange(0, len(ticks) * 4, 4), ticks, fontsize=16)

  # set the limit for x axis
  plt.xlim(-3, len(ticks)*4+0.5)
  plt.ylim(np.min(X)-0.05*(np.max(X)-np.min(X)), np.max(X)+0.00*(np.max(X)-np.min(X)))

  # set the title
  plt.ylabel(title, fontsize=18)

  maxcom = 0
  for j in range(Xu.shape[1]):
    data = []
    data.append(Xu[:Co, j])
    data.append(Xu[Co:No, j])
    data.append(Xu[No:Mi, j])
    data.append(Xu[Mi:Mo, j])
    data.append(Xu[Mo:Se, j])

    significant_combinations = []
    ls = list(range(1, len(data) + 1))
    combinations = [(ls[x], ls[x + y]) for y in reversed(ls) for x in range((len(ls) - y))]
    # print(combinations)
    for c in combinations:
        data1 = np.array(data[c[0] - 1]).astype(float)
        data2 = np.array(data[c[1] - 1]).astype(float)
        # Significance
        U, p = stats.mannwhitneyu(data1, data2, alternative='two-sided')
        significant_combinations.append([c, p])
        maxcom = max(len(significant_combinations), maxcom)

    bottom, top = ax.get_ylim()
    yrange = top - bottom

    # Significance bars
    for i, significant_combination in enumerate(significant_combinations):
        # Columns corresponding to the datasets of interest
        x1 = j*4.0-1.4+(significant_combination[0][0]-1)*0.7
        x2 = j*4.0-1.4+(significant_combination[0][1]-1)*0.7
        # What level is this bar among the bars above the plot?
        level = len(significant_combinations) - i - 0.7
        # Plot the bar
        bar_height = (yrange * 0.07 * level) + top
        bar_tips = bar_height - (yrange * 0.01)
        plt.plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k')
        # Significance level
        p = significant_combination[1]
        if p < 0.0005:
            sig_symbol = '***'
            text_gap = 0.0
        elif p < 0.005:
            sig_symbol = '**'
            text_gap = 0.0
        elif p < 0.05:
            sig_symbol = '*'
            text_gap = 0.0
        else:
            sig_symbol = 'ns'
            text_gap = 0.02
        text_height = bar_height - (yrange * 0.01)
        plt.text((x1 + x2) * 0.5, text_height + text_gap*yrange, sig_symbol, ha='center', c='k', fontsize=14)
  # Adjust y-axis
  ax.set_ylim(bottom - 0.02 * yrange, top + (yrange * 0.074 * (maxcom )))
  plt.yticks(fontsize=14)

  return

In [ ]:
def SVM_Binary_Classification_SFS_CV(XX, y, C, gamma):

    Paired_Groups = [[0, 1], [0, 2], [0, 34], [0, 234], [1, 2], [1, 34], [1, 234], [2, 3], [2, 34], [3, 4]]
    for pair in Paired_Groups:
        X = XX.copy()
        first_group = pair[0]
        second_group = pair[1]
        if second_group==234:
            Xpaired, ypaired = X[np.any([y == first_group, y == 2, y == 3, y == 4], axis=0)], y[np.any([y == first_group, y == 2, y == 3, y == 4], axis=0)]
            ypaired[ypaired==first_group]=0
            ypaired[ypaired==2]=1
            ypaired[ypaired==3]=1
            ypaired[ypaired==4]=1
        elif second_group==34:
            Xpaired, ypaired = X[np.any([y == first_group, y == 3, y == 4], axis=0)], y[np.any([y == first_group, y == 3, y == 4], axis=0)]
            ypaired[ypaired==first_group]=0
            ypaired[ypaired==3]=1
            ypaired[ypaired==4]=1
        else:
            Xpaired, ypaired = X[np.any([y == first_group, y == second_group], axis=0)], y[np.any([y == first_group, y == second_group], axis=0)]
            ypaired[ypaired==first_group]=0
            ypaired[ypaired==second_group]=1

        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=35)
        classifier = SVC(kernel='rbf', class_weight='balanced', probability=True, C=C, gamma=gamma)

        scaler = StandardScaler()
        Xpaired = scaler.fit_transform(Xpaired)

        sfs = SFS(classifier,
          k_features="best", # 'best', 'parsimonious'
          forward=True,
          floating=False,
          scoring='accuracy', # accuracy, f1, precision, recall, roc_auc
          cv=cv,
          n_jobs=-1)
        sfs = sfs.fit(Xpaired, ypaired)

        Xpaired = sfs.transform(Xpaired)

        tprs, aucs = {}, {}, {}
        mean_tpr, mean_auc, std_auc = {}, {}, {}, {}, {}
        tprs["micro"]=[]
        tprsTrain["micro"]=[]
        aucs["micro"]=[]
        mean_fpr = np.linspace(0, 1, 1000)
        Y_test_5fold, Y_pred_5fold = [], []
        Y_train_5fold, Y_trainpred_5fold = [], []

        for fold, (train, test) in enumerate(cv.split(Xpaired, ypaired)):
            scaler = StandardScaler()
            Xpaired[train] = scaler.fit_transform(Xpaired[train])
            Xpaired[test] = scaler.transform(Xpaired[test])
            classifier.fit(Xpaired[train], ypaired[train])
            y_score = classifier.predict_proba(Xpaired[test])
            y_score_Train = classifier.predict_proba(Xpaired[train])

            label_binarizer = LabelBinarizer().fit(ypaired[train])
            y_onehot_test = label_binarizer.transform(ypaired[test])

            fpr, tpr, roc_auc = {}, {}, {}
            fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_score[:,1:2].ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            interp_tpr = np.interp(mean_fpr, fpr["micro"], tpr["micro"])
            interp_tpr[0] = 0.0
            tprs["micro"].append(interp_tpr)
            aucs["micro"].append(roc_auc["micro"])

            fpr_grid = np.linspace(0.0, 1.0, 10000)

            pred_labels_te = classifier.predict(Xpaired[test])
            Y_test_5fold.extend(ypaired[test])
            Y_pred_5fold.extend(pred_labels_te)

        cnf_matrix = confusion_matrix(Y_test_5fold, Y_pred_5fold)

        tn, fp, fn, tp = cnf_matrix.ravel()
        Tpr = tp/(tp+fn)
        Tnr = tn/(tn+fp)
        Acc = (tp+tn)/(tp+fp+fn+tn)
        Acc = accuracy_score(Y_test_5fold, Y_pred_5fold)

        Mean_tpr["micro"] = np.mean(tprs["micro"], axis=0)
        Mean_tpr["micro"][-1] = 1.0
        mean_auc["micro"] = auc(mean_fpr, Mean_tpr["micro"])
        std_auc["micro"] = np.std(aucs["micro"])

        print(pair, '    sensitivity = ', format(100*Tpr, '.2f'), ' ,specificity = ', format(100*Tnr, '.2f'), '  ,accuracy = ', format(100*Acc, '.2f'), '  ,AUC = ', format(100*mean_auc["micro"], '.2f'))

    return

In [ ]:
def SVM_MultiClass_Classification_SFS_CV(X, y, C, gamma, title):

    target_names = np.array(['Control', 'NoDR', 'Mild', 'Moderate', 'Severe'])
    y = target_names[y]
    n_classes = len(np.unique(y))

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=35)
    classifier = SVC(kernel='rbf', class_weight='balanced', probability=True, C=C, gamma=gamma)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    sfs = SFS(classifier,
      k_features="best", # 'best', 'parsimonious'
      forward=True,
      floating=False,
      scoring='accuracy', # accuracy, f1, precision, recall, roc_auc
      cv=cv,
      n_jobs=-1)
    sfs = sfs.fit(X, y)

    X = sfs.transform(X)

    tprs, aucs, Mean_tpr, mean_auc, std_auc = {}, {}, {}, {}, {}
    tprs["micro"], tprs[0], tprs[1], tprs[2], tprs[3], tprs[4]= [], [], [], [], [], []
    aucs["micro"], aucs[0], aucs[1], aucs[2], aucs[3], aucs[4]= [], [], [], [], [], []
    mean_fpr = np.linspace(0, 1, 2000)
    Y_test_5fold, Y_pred_5fold = [], []

    for fold, (train, test) in enumerate(cv.split(X, y)):
        scaler = StandardScaler()
        X[train] = scaler.fit_transform(X[train])
        X[test] = scaler.transform(X[test])
        classifier.fit(X[train], y[train])
        y_score = classifier.predict_proba(X[test])

        label_binarizer = LabelBinarizer().fit(y[train])
        y_onehot_test = label_binarizer.transform(y[test])

        fpr, tpr, roc_auc = dict(), dict(), dict()
        fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        interp_tpr = np.interp(mean_fpr, fpr["micro"], tpr["micro"])
        interp_tpr[0] = 0.0
        tprs["micro"].append(interp_tpr)
        aucs["micro"].append(roc_auc["micro"])

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_onehot_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

            interp_tpr = np.interp(mean_fpr, fpr[i], tpr[i])
            interp_tpr[0] = 0.0
            tprs[i].append(interp_tpr)
            aucs[i].append(roc_auc[i])

        fpr_grid = np.linspace(0.0, 1.0, 1000)

        # Interpolate all ROC curves at these points
        mean_tpr = np.zeros_like(fpr_grid)
        for i in range(n_classes):
            mean_tpr += np.interp(fpr_grid, fpr[i], tpr[i])  # linear interpolation
        mean_tpr /= n_classes
        fpr["macro"] = fpr_grid
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        macro_roc_auc_ovr = roc_auc_score(y[test], y_score, multi_class="ovr", average="macro")

        pred_labels_tr = classifier.predict(X[train])
        pred_labels_te = classifier.predict(X[test])
        Y_test_5fold.extend(y[test])
        Y_pred_5fold.extend(pred_labels_te)

    cnf_matrix = confusion_matrix(Y_test_5fold, Y_pred_5fold)

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy for each class
    ACC = (TP+TN)/(TP+FP+FN+TN)

    Mean_tpr["micro"] = np.mean(tprs["micro"], axis=0)
    Mean_tpr["micro"][-1] = 1.0
    mean_auc["micro"] = auc(mean_fpr, Mean_tpr["micro"])
    std_auc["micro"] = np.std(aucs["micro"])

    for i in range(n_classes):
        Mean_tpr[i] = np.mean(tprs[i], axis=0)
        Mean_tpr[i][-1] = 1.0
        mean_auc[i] = auc(mean_fpr, Mean_tpr[i])
        std_auc[i] = np.std(aucs[i])

    print('sensitivity = ', format(100*np.mean(TPR), '.2f'), ' ,specificity = ', format(100*np.mean(TNR), '.2f'), '  ,accuracy = ', format(100*np.mean(ACC), '.2f'), '  ,AUC = ', format(100*mean_auc["micro"], '.2f'))

    fig, ax = plt.subplots(figsize=(8, 8))
    fig.set_facecolor('white')
    plt.plot(mean_fpr, Mean_tpr["micro"], label=f"micro-average ROC curve (AUC = {mean_auc['micro']:.2f})",
        color="red", linewidth=3)
    plt.rc('figure', dpi=1200)

    colors = cycle(["green", "magenta", "aqua", "darkorange", "cornflowerblue"])
    for class_id, color in zip(range(n_classes), colors):
        plt.plot(mean_fpr, Mean_tpr[class_id], label=f"ROC curve for {target_names[class_id]} (AUC = {mean_auc[class_id]:.2f})", color=color)

    plt.plot([0, 1], [0, 1], "k--", label="ROC curve for chance level (AUC = 0.5)")
    plt.axis("square")
    plt.xlabel("False Positive Rate", fontsize=18)
    plt.ylabel("True Positive Rate", fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=12)
    plt.ylim([0, 1])
    plt.xlim([0, 1])
    plt.show()
    return